# Build data with raptor

In [1]:
import sys
sys.path.append("../..")
from datasets import load_dataset
from src.service.provider import ProviderService
provider = ProviderService()

In [2]:
DATA_REPO = "BroDeadlines/TEST.edu_tdt_proposition_data"
# SPLIT = "TEST.basic_index_TDT_clean"
SPLIT = "INDEX.medium_index_TDT_clean"
test_dataset = load_dataset(DATA_REPO, split= SPLIT)
test_dataset

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 344
})

In [3]:
rows = test_dataset.filter(lambda e: e['url'] == "https://tuvanhocduong.tdtu.edu.vn/News")
rows

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 144
})

## Learn

In [24]:
idx = 300
tmp = test_dataset[104]['content']
tmp

'\xa0 \xa0 \xa0 \xa0\xa0Về nguyên tắc, khi kết thúc thời gian đăng ký kế hoạch học tập (KHHT) thì sinh viên không được phép hủy môn đã đăng ký trong KHHT. Sinh viên chỉ được quyền hủy môn trong thời gian Giảng viên cố vấn\xa0 (GVCV) hỗ trợ sinh viên kiểm tra, xem, sửa kết quả đăng ký\xa0và phải làm đơn trình bày rõ lí do xin hủy. Hoặc trong các trường hợp: trùng thời khóa biểu, vướng điều kiện môn học trước, môn học tiên quyết, lớp đầy…Do đó, sinh viên cần cân nhắc, suy nghĩ kỹ khi đăng ký KHHT.'

In [11]:
gemini = provider.get_simple_gemini_pro(model='gemini-1.5-flash-latest')

In [25]:
SUMMARIZATION_TEMPLATE = """Hãy tóm tắt văn bản sau một cách ngắn gọn và chi tiết.
Văn bản:
{context}
"""
prompt = SUMMARIZATION_TEMPLATE.format(context=tmp)

print(prompt)

Hãy tóm tắt văn bản sau một cách ngắn gọn và chi tiết.
Văn bản:
        Về nguyên tắc, khi kết thúc thời gian đăng ký kế hoạch học tập (KHHT) thì sinh viên không được phép hủy môn đã đăng ký trong KHHT. Sinh viên chỉ được quyền hủy môn trong thời gian Giảng viên cố vấn  (GVCV) hỗ trợ sinh viên kiểm tra, xem, sửa kết quả đăng ký và phải làm đơn trình bày rõ lí do xin hủy. Hoặc trong các trường hợp: trùng thời khóa biểu, vướng điều kiện môn học trước, môn học tiên quyết, lớp đầy…Do đó, sinh viên cần cân nhắc, suy nghĩ kỹ khi đăng ký KHHT.



In [26]:
a = gemini(prompt)
a

'Sinh viên không được phép hủy môn học đã đăng ký trong kế hoạch học tập (KHHT) sau khi kết thúc thời gian đăng ký, trừ trường hợp:\n\n* Được GVCV hỗ trợ kiểm tra, xem, sửa kết quả đăng ký và làm đơn trình bày lý do xin hủy.\n* Trùng thời khóa biểu, vướng điều kiện môn học trước, môn học tiên quyết, lớp đầy.\n\nDo đó, sinh viên cần cân nhắc kỹ trước khi đăng ký KHHT. \n'

## Update the splits

In [4]:
dataset = test_dataset.remove_columns(['split', 'splits', 'shards'])
dataset

Dataset({
    features: ['content', 'url', 'doc_id', 'propositions', 'proposition_list'],
    num_rows: 344
})

In [7]:
from src.utils.text_utils import create_splitter, try_split_texts
from src.chain.proposition import create_proposition_chain

contents = dataset['content']
splitter = create_splitter(chunk_size=1500, overlap=0, separators=[".", "\n\n", "\n", " "])
db, dist,lens = try_split_texts(texts=contents, splitter=splitter)

From 344 texts -> 499
Distribution: Counter({1: 255, 2: 53, 3: 21, 4: 8, 5: 3, 8: 2, 6: 2})
== About the splits ==
The mean of the splits is: 1.4505813953488371
The median of the splits is: 1.0
The max of the split is: 8
The min of the split is: 1
== About the splits length ==
The max of the split length is: 1478
The min of the split length is: 21
The mean of the split length is: 756.496993987976
The median of the split length is: 832.0


In [7]:
max_idx = lens.argmax()
lens[max_idx]
txt = db[max_idx].page_content
print(txt)

Kỹ thuật hóa học | Tuyển sinh Nhảy đến nội dung x Tuyển sinh English Main navigation Đại học Sau đại học Tư vấn Open Day 2024 Giảng viên/Viên chức  Sinh viên  Cựu sinh viên Kỹ thuật hóa học Kỹ thuật hóa học Đăng ký xét tuyển Đặt câu hỏi Kỹ thuật hoá học là ngành học giao thoa giữa khoa học và kỹ thuật, ứng dụng kiến thức về hoá lý, vật liệu vô cơ, vật liệu hữu cơ, tách chiết hợp chất, … vào sản xuất để phát triển và vận hành các quy trình công nghệ có liên quan đến hóa chất và công nghệ.  Sinh viên sau khi tốt nghiệp ngành Kỹ thuật hoá học có thể đảm nhiệm các vị trí như: chuyên viên vận hành và quản lý sản xuất; chuyên viên nghiên cứu phát triển sản phẩm; chuyên viên tư vấn quản lí và chuyển giao công nghệ; nhân viên kinh doanh hóa chất; nhân viên kiểm soát chất lượng sản phẩm tại các công ty, nhà máy sản xuất dược phẩm, mỹ phẩm, hương liệu, phân bón, vật liệu xây dựng, gốm sứ silicat, vật liệu polymer, vật liệu nano, vật liệu bán dẫn, hóa chất, cao su, vải, thuốc nhuộm, chất tẩy r

In [8]:
def create_split(row):
    docs = splitter.split_text(row['content'])
    return {**row, "splits": docs}
    
new_ds = dataset.map(create_split)
new_ds

Dataset({
    features: ['content', 'url', 'doc_id', 'propositions', 'proposition_list', 'splits'],
    num_rows: 344
})

In [9]:
a = new_ds[10]['splits']
len(a)

1

## Build RAPTOR tree

In [10]:
from src.rag.raptor import RAPTOR
raptor = RAPTOR(provider, reduce_method="proposition")

# vec_idx = "vec-raptor-basic_index_tdt_clean"
# text_idx = "text-raptor-basic_index_tdt_clean"

text_idx = "text-raptor-proposition-medium_index_tdt_vi"
vec_idx = "vec-raptor-proposition-medium_index_tdt_vi"

# text_idx = "text-raptor-medium_index_tdt"
# vec_idx = "vec-raptor-medium_index_tdt"

es = provider.get_elasticsearch_store(vec_idx)
bm = provider.get_elasticsearch_bm25(text_idx)

Create proposition chain


In [13]:
leaf_ids = []
leaf_texts = []

def collect_data123(row):
    docs = row['splits']
    for d in docs:
        leaf_ids.append(row['doc_id'])
        leaf_texts.append(d)
    return row

a = new_ds.map(collect_data123)
print(len(leaf_texts) ,"-", len(leaf_ids))
a

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

1302 - 1302


Dataset({
    features: ['content', 'url', 'doc_id', 'propositions', 'proposition_list', 'splits'],
    num_rows: 344
})

In [14]:
print(len(leaf_texts) ,"-", len(leaf_ids))

1302 - 1302


In [14]:
raptor.model = provider.get_simple_gemini_pro("gemini-1.5-flash-latest")
# raptor.embd = provider.get_gemini_embeddings("models/embedding-001")

In [13]:
raptor.test_log("Hello")

raptor.log - ERROR - Hello
NoneType: None


In [15]:
results = raptor.recursive_embed_cluster_summarize(leaf_texts, leaf_ids, level=1, n_levels=3)

--Generated 224 clusters--


Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


--Generated 44 clusters--
--Generated 10 clusters--


### Save pickle

In [16]:
import pickle

with open('tree_prop_raptor_vi.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)


### Up to Hugging Face

In [17]:
import pandas as pd

def assign_level_id(raptor_tree):
    levels = raptor_tree.keys()
    for level in levels:
        raptor_tree[level][0]['level_id'] = f"tree_{level}"
        raptor_tree[level][1]['level_id'] = f"tree_{level}"
    return

def print_raptor_info(results):
    print(results.keys())
    for i in results.keys():
        print("=======")
        print(len(results[i]))
        print(results[i][0].keys())
        print("->",results[i][0].shape)
        print(results[i][0]['level_id'].unique())
        print(results[i][1].keys())
        print("->",results[i][1].shape)
    return

def flatten_tree(raptor_tree):
    clusters = []
    summarizes = []
    for level in raptor_tree.keys():
        clusters.append(raptor_tree[level][0])
        summarizes.append(raptor_tree[level][1])
    return pd.concat(clusters), pd.concat(summarizes)

In [18]:
assign_level_id(results)
print_raptor_info(results)
clusters, summarizes = flatten_tree(results)

dict_keys([1, 2, 3])
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (1302, 5)
['tree_1']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (224, 5)
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (224, 5)
['tree_2']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (44, 5)
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (44, 5)
['tree_3']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (10, 5)


In [19]:
from datasets import Dataset
cluster_dataset = Dataset.from_pandas(clusters)
print(cluster_dataset)
summerize_dataset = Dataset.from_pandas(summarizes)
summerize_dataset

Dataset({
    features: ['text', 'embd', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 1570
})


Dataset({
    features: ['summaries', 'level', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 278
})

In [20]:
CLUSTER_REPO = "BroDeadlines/TEST.PART_CLUSTER.raptor.edu_tdt_data"
SUMMERIZE_REPO = "BroDeadlines/TEST.PART_SUMMERIZE.raptor.edu_tdt_data"
SPLIT = "TEST.medium_tdt_proposition_raptor_vi"

In [21]:
cluster_dataset.push_to_hub(CLUSTER_REPO, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/834 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.PART_CLUSTER.raptor.edu_tdt_data/commit/7085178c7be47471a183aeade82268c18c9d7f51', commit_message='Upload dataset', commit_description='', oid='7085178c7be47471a183aeade82268c18c9d7f51', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
summerize_dataset.push_to_hub(SUMMERIZE_REPO, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/970 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.PART_SUMMERIZE.raptor.edu_tdt_data/commit/8a4f9f6046ec82eb23edffe3215c7b3254522656', commit_message='Upload dataset', commit_description='', oid='8a4f9f6046ec82eb23edffe3215c7b3254522656', pr_url=None, pr_revision=None, pr_num=None)

### Up to ES

In [23]:
print_raptor_info(results)

dict_keys([1, 2, 3])
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (1302, 5)
['tree_1']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (224, 5)
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (224, 5)
['tree_2']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (44, 5)
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (44, 5)
['tree_3']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (10, 5)


In [24]:
from src.utils.type_utils import create_langchain_doc
# Initialize all_texts with leaf_texts
all_texts = [create_langchain_doc(txt, {"doc_id": id}) for txt, id in zip(leaf_texts, leaf_ids)]
all_texts[0]

Document(page_content='Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến, chọn học kỳ muốn gia hạn học phí, chọn loại đơn cần nộp và tiến hành điền các thông tin cần thiết. Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không. Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\n\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí, đính kèm minh chứng và nộp về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.', metadata={'doc_id': '573ca174-1b6d-11ef-a755-d38426455a06'})

In [31]:
1302 + 224 + 44 +10

1580

In [26]:

# Iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    d_ids = results[level][1]["doc_ids"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend([create_langchain_doc(txt, {"doc_id": id}) for txt, id in zip(summaries, d_ids)])


In [32]:
len(all_texts)

1580

In [27]:
print(vec_idx)
print(text_idx)

vec-raptor-proposition-medium_index_tdt_vi
text-raptor-proposition-medium_index_tdt_vi


In [33]:
b = bm.add_documents(all_texts)

ConnectionTimeout: Connection timed out

In [33]:
with open("bm_prop_ids_vi.txt", "w") as f:
    f.write("\n".join(b))

In [34]:
a = es.add_documents(all_texts)

ConnectionTimeout: Connection timed out

In [35]:
with open("es_ids_vi.txt", "w") as f:
    f.write("\n".join(a))

## Update shards

In [36]:
print(len(all_texts))
all_texts[0]

1580


Document(page_content='Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến, chọn học kỳ muốn gia hạn học phí, chọn loại đơn cần nộp và tiến hành điền các thông tin cần thiết. Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không. Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\n\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí, đính kèm minh chứng và nộp về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.', metadata={'doc_id': '573ca174-1b6d-11ef-a755-d38426455a06'})

In [37]:
def build_shards(all_data):
    shards_map = {}
    shards = {}
    for doc in all_data:
        d_id = doc.metadata['doc_id']
        # update new record
        if d_id not in shards_map:
            shards_map[d_id] = 1
            shards[d_id] = 1
        else:
            shards_map[d_id] += 1
            shards[d_id] += 1
        # update sub-tree
        if ";" not in d_id:
            continue
        keys = shards_map.keys()
        for k in keys:
            if k in d_id:
                shards_map[k] += 1
    return shards, shards_map

shards, hard_shards = build_shards(all_texts)
print(sum(shards.values()))
print(sum(hard_shards.values()))

1580
20923


In [38]:
print(cluster_dataset)
summerize_dataset

Dataset({
    features: ['text', 'embd', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 1523
})


Dataset({
    features: ['summaries', 'level', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 277
})

In [38]:
def update_shards(row):
    row['easy_shards'] = shards[row['doc_ids']]
    row['hard_shards'] = hard_shards[row['doc_ids']]
    return row
    
cluster_dataset_one = cluster_dataset.map(update_shards)
summerize_dataset_one = summerize_dataset.map(update_shards)

Map:   0%|          | 0/1570 [00:00<?, ? examples/s]

Map:   0%|          | 0/278 [00:00<?, ? examples/s]

In [39]:
summerize_dataset_one = summerize_dataset_one.remove_columns(['__index_level_0__'])
cluster_dataset_one = cluster_dataset_one.remove_columns(['__index_level_0__'])
print(cluster_dataset_one)
summerize_dataset_one

Dataset({
    features: ['text', 'embd', 'cluster', 'doc_ids', 'level_id', 'easy_shards', 'hard_shards'],
    num_rows: 1570
})


Dataset({
    features: ['summaries', 'level', 'cluster', 'doc_ids', 'level_id', 'easy_shards', 'hard_shards'],
    num_rows: 278
})

In [40]:
CLUSTER_REPO = "BroDeadlines/TEST.NEW.PART_CLUSTER.raptor.edu_tdt_data"
SUMMERIZE_REPO = "BroDeadlines/TEST.NEW.PART_SUMMERIZE.raptor.edu_tdt_data"
SPLIT = "TEST.medium_tdt_proposition_raptor_vi"

cluster_dataset_one.push_to_hub(CLUSTER_REPO, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.NEW.PART_CLUSTER.raptor.edu_tdt_data/commit/b5777a93c53b9b3f938b98f3e200d0a1deeb73b0', commit_message='Upload dataset', commit_description='', oid='b5777a93c53b9b3f938b98f3e200d0a1deeb73b0', pr_url=None, pr_revision=None, pr_num=None)

In [41]:
summerize_dataset_one.push_to_hub(SUMMERIZE_REPO, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/857 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.NEW.PART_SUMMERIZE.raptor.edu_tdt_data/commit/2a4c4f5100f953c5a8cff08b4ae126a412a8ddc5', commit_message='Upload dataset', commit_description='', oid='2a4c4f5100f953c5a8cff08b4ae126a412a8ddc5', pr_url=None, pr_revision=None, pr_num=None)

In [2]:
SUM_REPO = "BroDeadlines/TEST.PART_SUMMERIZE.raptor.edu_tdt_data"
CLU_REPO = "BroDeadlines/TEST.PART_CLUSTER.raptor.edu_tdt_data"
SPLIT = "TEST.basic_tdt_raptor"
sum_dataset = load_dataset(SUM_REPO, split= SPLIT)
sum_dataset

Generating TEST.basic_tdt_raptor split:   0%|          | 0/19 [00:00<?, ? examples/s]

Dataset({
    features: ['summaries', 'level', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 19
})

In [3]:
CLU_REPO = "BroDeadlines/TEST.PART_CLUSTER.raptor.edu_tdt_data"
SPLIT = "TEST.basic_tdt_raptor"
cluster_dataset = load_dataset(CLU_REPO, split= SPLIT)
cluster_dataset

Generating TEST.basic_tdt_raptor split:   0%|          | 0/117 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'embd', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 117
})

## Done